# T1530 - Data from Cloud Storage
Adversaries may access data from cloud storage.

Many IaaS providers offer solutions for online data object storage such as Amazon S3, Azure Storage, and Google Cloud Storage. Similarly, SaaS enterprise platforms such as Office 365 and Google Workspace provide cloud-based document storage to users through services such as OneDrive and Google Drive, while SaaS application providers such as Slack, Confluence, Salesforce, and Dropbox may provide cloud storage solutions as a peripheral or primary use case of their platform. 

In some cases, as with IaaS-based cloud storage, there exists no overarching application (such as SQL or Elasticsearch) with which to interact with the stored objects: instead, data from these solutions is retrieved directly though the [Cloud API](https://attack.mitre.org/techniques/T1059/009). In SaaS applications, adversaries may be able to collect this data directly from APIs or backend cloud storage objects, rather than through their front-end application or interface (i.e., [Data from Information Repositories](https://attack.mitre.org/techniques/T1213)). 

Adversaries may collect sensitive data from these cloud storage solutions. Providers typically offer security guides to help end users configure systems, though misconfigurations are a common problem.(Citation: Amazon S3 Security, 2019)(Citation: Microsoft Azure Storage Security, 2019)(Citation: Google Cloud Storage Best Practices, 2019) There have been numerous incidents where cloud storage has been improperly secured, typically by unintentionally allowing public access to unauthenticated users, overly-broad access by all users, or even access for any anonymous person outside the control of the Identity Access Management system without even needing basic user permissions.

This open access may expose various types of sensitive data, such as credit cards, personally identifiable information, or medical records.(Citation: Trend Micro S3 Exposed PII, 2017)(Citation: Wired Magecart S3 Buckets, 2019)(Citation: HIPAA Journal S3 Breach, 2017)(Citation: Rclone-mega-extortion_05_2021)

Adversaries may also obtain then abuse leaked credentials from source repositories, logs, or other means as a way to gain access to cloud storage objects.

## Atomic Tests

### Atomic Test #1 - Azure - Enumerate Azure Blobs with MicroBurst
Upon successful execution, this test will utilize a wordlist to enumerate the public facing containers and blobs of a specified Azure storage account. 
See https://www.netspi.com/blog/technical/cloud-penetration-testing/anonymously-enumerating-azure-file-resources/ . 

**Supported Platforms:** iaas:azure
#### Dependencies:  Run with `powershell`!
##### Description: The Invoke-EnumerateAzureBlobs module must exist in PathToAtomicsFolder\..\ExternalPayloads.

##### Check Prereq Commands:
```powershell
if (test-path "PathToAtomicsFolder\..\ExternalPayloads\Invoke-EnumerateAzureBlobs.ps1"){exit 0} else {exit 1}

```
##### Get Prereq Commands:
```powershell
New-Item -Type Directory "PathToAtomicsFolder\..\ExternalPayloads\" -ErrorAction Ignore -Force | Out-Null
invoke-webrequest "https://raw.githubusercontent.com/NetSPI/MicroBurst/156c4e9f4253b482b2b68eda4651116b9f0f2e17/Misc/Invoke-EnumerateAzureBlobs.ps1" -outfile "PathToAtomicsFolder\..\ExternalPayloads\Invoke-EnumerateAzureBlobs.ps1"

```
##### Description: The wordlist file for search permutations must exist in PathToAtomicsFolder\..\ExternalPayloads.

##### Check Prereq Commands:
```powershell
if (test-path "PathToAtomicsFolder\..\ExternalPayloads\permutations.txt"){exit 0} else {exit 1}

```
##### Get Prereq Commands:
```powershell
invoke-webrequest "https://raw.githubusercontent.com/NetSPI/MicroBurst/156c4e9f4253b482b2b68eda4651116b9f0f2e17/Misc/permutations.txt" -outfile "PathToAtomicsFolder\..\ExternalPayloads\permutations.txt"

```

In [ ]:
Invoke-AtomicTest T1530 -TestNumbers 1 -GetPreReqs

#### Attack Commands: Run with `powershell`
```powershell
import-module "PathToAtomicsFolder\..\ExternalPayloads\Invoke-EnumerateAzureBlobs.ps1"
Invoke-EnumerateAzureBlobs -base secure -permutations "PathToAtomicsFolder\..\ExternalPayloads\permutations.txt" -outputfile "$env:temp\T1530Test1.txt"
```

In [ ]:
Invoke-AtomicTest T1530 -TestNumbers 1

#### Cleanup: 
```powershell
remove-item $env:temp\T1530Test1.txt -erroraction silentlycontinue
```

In [ ]:
Invoke-AtomicTest T1530 -TestNumbers 1 -Cleanup

### Atomic Test #2 - Azure - Scan for Anonymous Access to Azure Storage (Powershell)Upon successful execution, this test will test for anonymous access to Azure storage containers by invoking a web request and outputting the results to a file. 
The corresponding response could then be interpreted to determine whether or not the resource/container exists, as well as other information. 
See https://ninocrudele.com/the-three-most-effective-and-dangerous-cyberattacks-to-azure-and-countermeasures-part-2-attack-the-azure-storage-service     
**Supported Platforms:** iaas:azure#### Attack Commands: Run with `powershell`
```powershell
try{$response = invoke-webrequest "https://T1530Test2.blob.core.windows.net/None/None" -method "GET"}
catch [system.net.webexception]
{if($_.Exception.Response -ne $null)
{$Response = $_.Exception.Response.GetResponseStream()
$ReadResponse = New-Object System.IO.StreamReader($Response)
$ReadResponse.BaseStream.Position = 0
$responseBody = $ReadResponse.ReadToEnd()}
else {$responseBody = "The storage account could not be anonymously accessed."}}
"Response received for T1530Test2.blob.core.windows.net/None/None: $responsebody" | out-file -filepath $env:temp\T1530Test2.txt -append
```

In [ ]:
Invoke-AtomicTest T1530 -TestNumbers 2

#### Cleanup: 
```powershell
remove-item $env:temp\T1530Test2.txt -erroraction silentlycontinue
```

In [ ]:
Invoke-AtomicTest T1530 -TestNumbers 2 -Cleanup

### Atomic Test #3 - AWS - Scan for Anonymous Access to S3
Upon successful execution, this test will test for anonymous access to AWS S3 buckets and dumps all the files to a local folder. 

**Supported Platforms:** iaas:aws

Elevation Required (e.g. root or admin)
#### Dependencies:  Run with `sh`!
##### Description: Check if ~/.aws/credentials file has a default stanza is configured

##### Check Prereq Commands:
```sh
cat ~/.aws/credentials | grep "default"
aws s3api create-bucket --bucket redatomic-test2
aws s3api put-bucket-policy --bucket redatomic-test2 --policy file://$PathToAtomicsFolder/T1530/src/policy.json
touch /tmp/T1530.txt
aws s3 cp /tmp/T1530.txt s3://redatomic-test2

```
##### Get Prereq Commands:
```sh
echo Please install the aws-cli and configure your AWS default profile using: aws configure

```

In [ ]:
Invoke-AtomicTest T1530 -TestNumbers 3 -GetPreReqs

#### Attack Commands: Run with `sh`
```sh
aws --no-sign-request s3 cp --recursive s3://redatomic-test2 /tmp/redatomic-test2
```

In [ ]:
Invoke-AtomicTest T1530 -TestNumbers 3

#### Cleanup: 
```sh
aws s3 rb s3://redatomic-test2 --force 
rm -rf /tmp/redatomic-test2
```

In [ ]:
Invoke-AtomicTest T1530 -TestNumbers 3 -Cleanup

## Detection
Monitor for unusual queries to the cloud provider's storage service. Activity originating from unexpected sources may indicate improper permissions are set that is allowing access to data. Additionally, detecting failed attempts by a user for a certain object, followed by escalation of privileges by the same user, and access to the same object may be an indication of suspicious activity.


## Shield Active Defense
### Pocket Litter 
 Place data on a system to reinforce the legitimacy of the system or user. 

 Pocket Litter is data placed on a system to convince an adversary that the system and users are real.  Pocket litter includes documents, registry entries, log history, browsing history, connection history, and other user data that one would expect to exist on a user's computer.  This content may overlap with Decoy Content, however Pocket Litter covers aspects beyond just content (e.g.: Installed Applications, source code, clutter on a system, etc.).
#### Opportunity
In an adversary engagement scenario, there is an opportunity to seed content to influence an adversary's behaviors, test their interest in specific topics, or add legitimacy to a system or environment.
#### Use Case
A defender can stage a variety of pocket litter files on an attached storage space. This data may include topics that align to a persona, topics an adversary is interested in, etc.
#### Procedures
When staging a decoy system and user account, populate a user's folders and web history to make it look realistic to an adversary.
Stage a USB device with documents on a specific topic in order to see if they are exfiltrated by an adversary.
